In [161]:
import pandas as pd
import numpy as np
import plotly as pt
import seaborn as sns
!pip install pymatgen
!pip install mp_api
import requests
import json

In [162]:
extract_CIFs = False
cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"

In [163]:
update_IDs = False  #request material IDs from databases
target_col = 'Log_rate'
cross_validation = True
dataset_from_google_drive = True

df = None
if(dataset_from_google_drive):
  df = pd.read_excel("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/Perovskite dataset export.xlsx",sheet_name='Photocatalytic dataset')
else:
  df = pd.read_excel("dataset.xlsx")

In [164]:
df

,Perovskite,Hill formula,Interlayer space composition,"Bandgap, eV",DOI,Materials Project ID,COD_ID,Springer_ID,MP_CIF_modifier,COD_CIF_modifier,...,"CatW, g/L","Alcohol, %",Sac Agent 1,Sac agent 2,Light type,"Power, W","Wave length min, nm",Cell material,"Temperature, K","Rate, umol/(g*h)"
0,K4Nb6O17,K4 Nb6 O17,NaN,3.50,10.1007/BF00765319,mp-560692,1001842,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,130.000000
1,KLaNb2O7,K La Nb2 O7,NaN,3.20,10.1007/BF00765319,mp-1223501,1545643,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,28.000000
2,RbLaNb2O7,La Nb2 O7 Rb,NaN,3.35,10.1007/BF00765319,mp-553965,-1,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,60.000000
3,CsLaNb2O7,Cs La Nb2 O7,NaN,3.30,10.1007/BF00765319,mp-553248,2004917,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,12.000000
4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,3.35,10.1007/BF00765319,mp-557195,1521061,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,38.181818
1085,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,36.363636
1086,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,9.090909
1087,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,9.090909


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            1089 non-null   object 
 1   Hill formula                          730 non-null    object 
 2   Interlayer space composition          5 non-null      object 
 3   Bandgap, eV                           1054 non-null   float64
 4   DOI                                   1084 non-null   object 
 5   Materials Project ID                  1058 non-null   object 
 6   COD_ID                                935 non-null    object 
 7   Springer_ID                           257 non-null    object 
 8   MP_CIF_modifier                       510 non-null    object 
 9   COD_CIF_modifier                      197 non-null    object 
 10  Springer_CIF_modifier                 121 non-null    object 
 11  Materials Project

In [166]:
df.describe()

,"Bandgap, eV",Z,Z_MP,Z_COD,Z_Springer,"d,A",a_MP,b_MP,c_MP,a_COD,...,Oxygen_concentration avg,Packing fraction avg,CalcT(K),Calc time (h),"CatW, g/L","Alcohol, %","Power, W","Wave length min, nm","Temperature, K","Rate, umol/(g*h)"
count,1054.000000,694.000000,832.000000,836.000000,836.000000,807.000000,827.000000,354.000000,354.000000,832.000000,...,832.000000,832.000000,1010.000000,973.000000,987.000000,966.000000,994.000000,689.000000,540.000000,989.000000
mean,3.313241,2.275216,1.260817,1.281100,0.151715,13.707422,3.084536,7.406617,14.790353,3.040421,...,inf,0.238767,1287.020792,19.338386,1.428618,5.628230,306.726358,288.388970,294.435185,6914.999659
std,0.644047,1.762830,3.460696,2.116075,0.642689,8.610255,4.236046,3.399894,8.264087,6.154639,...,NaN,0.257556,222.542464,25.264723,1.324191,8.781436,198.606250,88.814702,13.175195,21648.175384
min,0.010000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,3.840000,3.904344,0.000000,...,0.000000,0.000000,453.000000,0.000000,0.000000,0.000000,100.000000,200.000000,288.000000,0.000000
25%,3.100000,1.000000,0.000000,0.000000,0.000000,10.548300,0.000000,3.950634,10.502783,0.000000,...,0.037279,0.000000,1223.000000,10.000000,0.500000,0.000000,125.000000,220.000000,288.000000,15.500000
50%,3.410000,2.000000,0.000000,0.000000,0.000000,14.733000,0.000000,7.415443,12.879040,0.000000,...,0.042938,0.000000,1373.000000,12.000000,1.000000,1.000000,300.000000,220.000000,288.000000,166.000000
75%,3.600000,4.000000,1.000000,2.000000,0.000000,17.900000,5.801718,7.938952,15.513552,3.899000,...,0.046684,0.533675,1423.000000,24.000000,2.000000,10.000000,450.000000,400.000000,293.000000,1680.000000
max,5.500000,16.000000,50.000000,16.000000,4.000000,38.670000,22.855297,20.551678,48.809404,41.857000,...,inf,0.657822,1873.000000,200.000000,10.000000,90.000000,1000.000000,660.000000,333.000000,156000.000000


In [167]:
df.columns

Index(['Perovskite', 'Hill formula', 'Interlayer space composition',
       'Bandgap, eV', 'DOI', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'MP_CIF_modifier', 'COD_CIF_modifier', 'Springer_CIF_modifier',
       'Materials Project verification', 'COD verification',
       'Springer verification', 'General verification', 'MP_CIF_modified',
       'COD_CIF_modified', 'Springer_CIF_modified', 'Z', 'Z_MP', 'Z_COD',
       'Z_Springer', 'a, A', 'b, A', 'c, A', 'Symmetry group', 'd,A', 'a_MP',
       'b_MP', 'c_MP', 'a_COD', 'b_COD', 'c_COD', 'a_Springer', 'b_Springer',
       'c_Springer', 'Number of octahedrons on a layer', 'Valence electrons',
       'Volume', 'Volume_MP', 'Volume_COD', 'Volume_Springer',
       'Valence Electrons Density', 'Valence Electrons Density_MP',
       'Valence Electrons Density_COD', 'Springer_Valence Electrons Density',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       

In [168]:
init_columns = ['Perovskite', 'Interlayer space composition',
       'Bandgap, eV', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'Z', 'a, A', 'b, A', 'c, A',
       'Symmetry group', 'd,A',
       'Number of octahedrons on a layer', 'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'Nitrogen', 'Promotion method', 'Promoter', 'Promoter, w%',
       'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Sac Agent 1',
       'Sac agent 2', 'Light type', 'Power, W', 'Wave length min, nm',
       'Cell material', 'Temperature, K', 'Rate, umol/(g*h)']

init_columns = init_columns + ['MP_CIF_modifier', 'COD_CIF_modifier', 'Springer_CIF_modifier',
       'Materials Project verification', 'COD verification',
       'Springer verification', 'General verification', 'MP_CIF_modified',
       'COD_CIF_modified', 'Springer_CIF_modified']
print(init_columns)

['Perovskite', 'Interlayer space composition', 'Bandgap, eV', 'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A', 'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer', 'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen', 'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Sac Agent 1', 'Sac agent 2', 'Light type', 'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K', 'Rate, umol/(g*h)', 'MP_CIF_modifier', 'COD_CIF_modifier', 'Springer_CIF_modifier', 'Materials Project verification', 'COD verification', 'Springer verification', 'General verification', 'MP_CIF_modified', 'COD_CIF_modified', 'Springer_CIF_modified']


In [169]:
df = df[init_columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Perovskite                        1089 non-null   object 
 1   Interlayer space composition      5 non-null      object 
 2   Bandgap, eV                       1054 non-null   float64
 3   Materials Project ID              1058 non-null   object 
 4   COD_ID                            935 non-null    object 
 5   Springer_ID                       257 non-null    object 
 6   Z                                 694 non-null    float64
 7   a, A                              735 non-null    object 
 8   b, A                              717 non-null    object 
 9   c, A                              717 non-null    object 
 10  Symmetry group                    494 non-null    object 
 11  d,A                               807 non-null    float64
 12  Number

#Formatting


In [170]:
subs_map = {
    "Ph": "C6H5",
    "Bn": "C7H7",
    #"Pr": "C3H7",
    "Bu": "C4H9",
    "Hx": "C6H13",
    "Me": "CH3",
    "Et": "C2H5",
    "Oc": "C8H17",
    "Dc": "C10H21",
}

import re

def expand_substituents(formula):
    if pd.isna(formula):
        return formula

    for abbr, full in subs_map.items():
        formula = re.sub(rf'{abbr}', full, formula)
    return formula

In [171]:
print(df.shape[0])
df = df[~df['Perovskite'].str.contains("Nx", na=False)]
df = df[~df['Perovskite'].str.contains("Ox", na=False)]
print(df.shape[0])

1089
1076


In [172]:
df['Perovskite'] = df['Perovskite'].apply(expand_substituents)
#print(expand_substituents('HNdTiO4*MeNH2'))

In [173]:
df = df.replace("-", np.nan)
df["a, A"] = df["a, A"].astype("float64")
df["b, A"] = df["b, A"].astype("float64")
df["c, A"] = df["c, A"].astype("float64")
df["Number of octahedrons on a layer"] = df["Number of octahedrons on a layer"].astype("float64")

/tmp/ipython-input-3678714408.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [174]:
df["Nitrogen"] = df["Nitrogen"].fillna(False)
df["Nitrogen"]
df["Nitrogen"].astype("bool")
df["Nitrogen"].value_counts()

/tmp/ipython-input-1068474039.py:1: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,count
Nitrogen,
False,1029
True,47


In [175]:
df["Promoter, w%"] = df["Promoter, w%"].fillna(0)
df["Promoter, w%"].value_counts()

,count
"Promoter, w%",
0.000,493
1.000,295
0.100,81
0.500,72
0.150,27
0.200,24
0.300,16
3.000,15
2.000,10


In [176]:
df["Surface area, m2/g"] = df["Surface area, m2/g"].astype("float64")
df["Surface area, m2/g"].value_counts()

#df['Materials Project ID'] = df['Materials Project ID'].astype(str)
print(df['Materials Project ID'].dtype)

object


In [177]:
df["Perovskite"].value_counts()

,count
Perovskite,
HCa2Nb3O10,33
La2Ti2O7,32
H2Nd2Ti3O10*2C4H9NH2,32
TiO2,30
K4Nb6O17,28
...,...
Na2La2Mn0.9Ti2.1O10,1
Na2La2MnTi2O10,1
H0.4K1.6La2Ti3O10*1.07H2O,1


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1076 entries, 0 to 1088
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Perovskite                        1076 non-null   object 
 1   Interlayer space composition      5 non-null      object 
 2   Bandgap, eV                       1041 non-null   float64
 3   Materials Project ID              1045 non-null   object 
 4   COD_ID                            922 non-null    object 
 5   Springer_ID                       257 non-null    object 
 6   Z                                 694 non-null    float64
 7   a, A                              731 non-null    float64
 8   b, A                              713 non-null    float64
 9   c, A                              713 non-null    float64
 10  Symmetry group                    494 non-null    object 
 11  d,A                               796 non-null    float64
 12  Number of o

In [179]:
df.to_excel("checkpoint_formatting.xlsx")


#Data cleaning

##Missing data handling

In [180]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,1071
"Bandgap, eV",35
Materials Project ID,31
COD_ID,154
Springer_ID,819
Z,382
"a, A",345
"b, A",363
"c, A",363


In [181]:
df.columns

Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Sac Agent 1', 'Sac agent 2', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)', 'MP_CIF_modifier', 'COD_CIF_modifier',
       'Springer_CIF_modifier', 'Materials Project verification',
       'COD verification', 'Springer verification', 'General verification',
       'MP_CIF_modified', 'COD_CIF_modified', 'Springer_CIF_modified'],
      dtype='object')

In [182]:
drop_columns = ['Perovskite', 'Bandgap, eV',
       'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'CatW, g/L', 'Light type',
       'Power, W',
       'Rate, umol/(g*h)']

drop_columns=['Perovskite','Rate, umol/(g*h)'] #drop nan in these columns
df['Bandgap, eV'] = df['Bandgap, eV'].fillna(df['Bandgap, eV'].mean())
df['Prep Method'] = df['Prep Method'].fillna(df['Prep Method'].mode()[0])
df['CalcT(K)'] = df['CalcT(K)'].fillna(df['CalcT(K)'].mean())
df['Calc time (h)'] = df['Calc time (h)'].fillna(df['Calc time (h)'].mean())
df['CatW, g/L'] = df['CatW, g/L'].fillna(df['CatW, g/L'].mean())
df['Light type'] = df['Light type'].fillna(df['Light type'].mode()[0])
df['Power, W'] = df['Power, W'].fillna(df['Power, W'].mean())

df.dropna(subset=drop_columns, inplace=True)
df.shape[0]

976

In [183]:
df = df[df["General verification"] == False]
df.shape[0]

897

In [184]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,893
"Bandgap, eV",0
Materials Project ID,18
COD_ID,121
Springer_ID,675
Z,350
"a, A",318
"b, A",333
"c, A",333


In [185]:
def calculate_average_specific_area(dataframe):
  total=0;
  n=0;
  for index, row in df.iterrows():
    area =  row['Surface area, m2/g']
    if(np.isnan(area) or area > 55):
      continue
    total += area
    n+=1
  return total/n

In [186]:
avr_specific_area = calculate_average_specific_area(df)
print(avr_specific_area)
df["Surface area, m2/g"] = df["Surface area, m2/g"].fillna(avr_specific_area)

df["Alcohol, %"] = df["Alcohol, %"].fillna(0)
df["Wave length min, nm"] = df["Wave length min, nm"].fillna(df["Wave length min, nm"].min())
df["Cell material"] = df["Cell material"].fillna(df["Cell material"].mode()[0])
print(df["Cell material"].mode()[0])
df["Temperature, K"] = df["Temperature, K"].fillna(298.15)
df["Materials Project ID"] = df["Materials Project ID"].fillna(-1)
df["COD_ID"] = df["COD_ID"].fillna(-1)
df["Springer_ID"] = df["Springer_ID"].fillna(-1)

10.374370192307678
quartz


In [187]:
df["Promoter"] = df["Promoter"].fillna("No promoter")
df["Promotion method"] = df["Promotion method"].fillna("No promotion")

In [188]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,893
"Bandgap, eV",0
Materials Project ID,0
COD_ID,0
Springer_ID,0
Z,350
"a, A",318
"b, A",333
"c, A",333


## Duplicates handling

In [189]:
duplicates = df[ df.duplicated()]
duplicates

,Perovskite,Interlayer space composition,"Bandgap, eV",Materials Project ID,COD_ID,Springer_ID,Z,"a, A","b, A","c, A",...,MP_CIF_modifier,COD_CIF_modifier,Springer_CIF_modifier,Materials Project verification,COD verification,Springer verification,General verification,MP_CIF_modified,COD_CIF_modified,Springer_CIF_modified


##Handling outliers

In [190]:
columns_outliers_detedection = [ 'Bandgap, eV',
      'Calc time (h)', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %',
       'Power, W', 'Wave length min, nm', 'Temperature, K']
columns_outliers_detedection = [ 'Bandgap, eV',
      'Calc time (h)', 'Promoter, w%',
       'CatW, g/L',
       'Power, W', 'Wave length min, nm', 'Temperature, K']
#columns_outliers_detedection=[]

In [191]:
import plotly.express as px
def plot_destribution(df,column):
  #fig = px.histogram(df, x=column,nbins=40,width=800, height=600, title=column)
  fig = px.violin(df, x=column,width=600, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()
def plot_histogram(df,column):
  fig = px.histogram(df, x=column,nbins=20,width=600, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()

In [192]:
for column in columns_outliers_detedection:
  plot_histogram(df,column)

In [193]:
def IQR_column_outliers_removal(_df, _column):
  num_rows_before = _df.shape[0]
  Q1 = _df[_column].quantile(0.25)
  Q3 = _df[_column].quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  df_clean = _df[(_df[_column]>= lower_bound) & (_df[_column] <= upper_bound)]
  num_rows_after = df_clean.shape[0]
  removed = num_rows_before - num_rows_after
  print(f'Column: {_column} removed ouliers: {removed }')
  print(f'min: {lower_bound} max: {upper_bound}')
  return df_clean

In [194]:
for column in columns_outliers_detedection:
  df = IQR_column_outliers_removal(df,column)
print(f'Rows left: {df.shape[0]}')

Column: Bandgap, eV removed ouliers: 153
min: 2.6000000000000005 max: 4.2
Column: Calc time (h) removed ouliers: 98
min: -11.0 max: 45.0
Column: Promoter, w% removed ouliers: 25
min: -1.5 max: 2.5
Column: CatW, g/L removed ouliers: 24
min: -1.75 max: 4.25
Column: Power, W removed ouliers: 12
min: -287.5 max: 812.5
Column: Wave length min, nm removed ouliers: 120
min: 170.0 max: 250.0
Column: Temperature, K removed ouliers: 4
min: 272.77500000000003 max: 313.37499999999994
Rows left: 461


#Descriptors calculation



## Material Project IDs


In [195]:
from mp_api.client import MPRester

API_KEY = "GFsoU5OV3dEngGT860TOtWcn35bE4y6l"
with MPRester(API_KEY) as mpr:
    materials = mpr.materials.search(formula="KCa2Nb3O10")
for material in materials:
    print(material.material_id, material.formula_pretty)

Retrieving MaterialsDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

mp-557195 KCa2Nb3O10
mp-1223594 KCa2Nb3O10


In [196]:
def get_material_id_by_formula(f):
  with MPRester(API_KEY) as mpr:
    print(f"Composition: {f}")
    try:
      materials = mpr.materials.search(formula=f)
      if len(materials)>0:
        print(f"Composition: {f} ID is {materials[0].material_id}")
        return str(materials[0].material_id)
      else:
        print(f"Composition: {f} ID is not found")
        return str(-1)
    except Exception as e:
      print(f"Composition: {f} ID is not found")
      return str(-1)

In [197]:
def update_Materials_Project_IDs(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['Materials Project ID']) or row['Materials Project ID']==-1 or row['Materials Project ID']==-2 :
            new_id = get_material_id_by_formula(row['Perovskite'])
            print(f"New ID fetched: {new_id}  for formula {row['Perovskite']}")
            _df.at[index, 'Materials Project ID'] = str(new_id)
  return _df

In [198]:
if update_IDs:
  df = update_Materials_Project_IDs(df)

##HIll's formula

In [199]:
import re
from collections import Counter
def parse_formula(formula):
    """Extracts element counts from a chemical formula."""
    pattern = r"([A-Z][a-z]*)(\d*)"
    matches = re.findall(pattern, formula)

    element_counts = Counter()
    for element, count in matches:
        element_counts[element] += int(count) if count else 1  # Default to 1 if count is missing

    return element_counts
def to_hill_notation(formula):
    """Converts a chemical formula to Hill notation."""
    element_counts = parse_formula(formula)

    # Hill notation rules
    if "C" in element_counts:
        elements_sorted = ["C", "H"] if "H" in element_counts else ["C"]
        remaining_elements = sorted(e for e in element_counts if e not in ["C", "H"])
        elements_sorted.extend(remaining_elements)
    else:
        elements_sorted = sorted(element_counts.keys())
    hill_formula = " ".join(f"{el}{element_counts[el] if element_counts[el] > 1 else ''}" for el in elements_sorted)
    #print(f':{hill_formula}:')
    return hill_formula


In [200]:
formulas = ["H2O", "C6H12O6", "NH3", "Fe2O3", "CH4", "CCl4", "H4C", "KCa2Nb3O10"]
for f in formulas:
    print(f"{f} -> {to_hill_notation(f)}")

H2O -> H2 O
C6H12O6 -> C6 H12 O6
NH3 -> H3 N
Fe2O3 -> Fe2 O3
CH4 -> C H4
CCl4 -> C Cl4
H4C -> C H4
KCa2Nb3O10 -> Ca2 K Nb3 O10


In [201]:
df['Hill formula']=df['Perovskite'].apply(to_hill_notation)
df

,Perovskite,Interlayer space composition,"Bandgap, eV",Materials Project ID,COD_ID,Springer_ID,Z,"a, A","b, A","c, A",...,COD_CIF_modifier,Springer_CIF_modifier,Materials Project verification,COD verification,Springer verification,General verification,MP_CIF_modified,COD_CIF_modified,Springer_CIF_modified,Hill formula
9,TiO2,NaN,3.0,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,True,True,False,False,mp-1245098,1010942,NaN,O2 Ti
19,TiO2,NaN,3.0,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,True,True,False,False,mp-1245098,1010942,NaN,O2 Ti
69,RbLaTa2O7,NaN,3.9,mp-541600,-1,sd_1050391,1.0,3.885,3.885,11.12,...,NaN,Pr->La,True,False,True,False,mp-541600,-1,M_Springer6,La O7 Rb Ta2
70,RbPrTa2O7,NaN,3.5,-1,-1,sd_1050391,1.0,3.884,3.884,11.12,...,NaN,NaN,False,False,True,False,-1,-1,sd_1050391,O7 Pr Rb Ta2
71,RbNdTa2O7,NaN,3.8,-1,-1,sd_1050391,1.0,3.846,3.846,11.10,...,NaN,Pr->Nd,False,False,True,False,-1,-1,M_Springer7,Nd O7 Rb Ta2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,True,True,False,False,mp-1245098,1010942,NaN,O2 Ti
1085,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,True,True,False,False,mp-1245098,1010942,NaN,O2 Ti
1086,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,True,True,False,False,mp-1245098,1010942,NaN,O2 Ti
1087,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,True,True,False,False,mp-1245098,1010942,NaN,O2 Ti


##COD_IDs


In [202]:
def get_COD_ID_for_formula(formula):
  print(f"Formula :{formula}:")
  url=f"https://www.crystallography.net/cod/result?formula={formula}&format=json"
  response = requests.get(url)
  if response.status_code == 200:
    data = response.json()
    #print("Len: ",len(data))
    if(len(data)>0):
      #print(data[0]['file'])
      return data[0]['file']
  print(-1)
  return -1

def update_COD_ID(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['COD_ID']) or row['COD_ID']==-1 or row['COD_ID']==-2 :
            print(f"New ID fetching start for formula {row['Hill formula']}")
            new_id = get_COD_ID_for_formula(row['Hill formula'])
            print(f"New ID fetched: {new_id}  for formula {row['Hill formula']}")
            _df.at[index, 'COD_ID'] = new_id
  return _df


In [203]:
df.to_excel('checkpoint_Hill_formula.xlsx')
#print(get_COD_ID("O2 Ti"))
if update_IDs:
  df = update_COD_ID(df)

##CIF files extraction

In [204]:
def extract_CIF_from_MP(material_id):
  if pd.isna(material_id) or material_id == "-1" or material_id == -1 or material_id == -2:
     print("Skipping invalid ID:", material_id)
     return
  print("ID: ", material_id)

  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif'
  if os.path.exists(path):
    print(f'CIF file for {material_id} already exist')
    return

  with MPRester(API_KEY) as mpr:
    #data = mpr.materials.search(material_ids=material_id)
    structure = mpr.materials.get_structure_by_material_id(material_id )
    cif_string = structure.to("struct.cif")

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  with open(f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif', 'w') as f:
      f.write(cif_string)
  print("ID: ", material_id, " done!")

In [205]:
import os
def extract_cif_from_COD(COD_ID):
  if(COD_ID==-1 or COD_ID==-2 ):
    return
  print(COD_ID)
  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{COD_ID}.cif'
  if os.path.exists(path):
    print(f'CIF file for {COD_ID} already exist')
    return

  url = f"https://www.crystallography.net/cod/{COD_ID}.cif"
  response = requests.get(url)

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  if response.status_code == 200:
    print("Sucess")
    with open(path, 'w') as f:
        f.write(response.text)
  else:
    print('No CIF')

In [206]:
if extract_CIFs:
  for material_id in df['COD_ID'].unique():
    extract_cif_from_COD(material_id)

In [207]:
if extract_CIFs:
  for material_id in df['Materials Project ID'].unique():
    extract_CIF_from_MP(material_id)

##Valence electron discriptors

In [208]:
!pip install matminer
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import ValenceOrbital
from pymatgen.core.composition import Composition


In [209]:
df['composition_obj'] = df['Hill formula'].apply(Composition)
print("Null compositions",df['composition_obj'].isna().sum())
vo_feat = ValenceOrbital()
df = vo_feat.featurize_dataframe(df, col_id='composition_obj')
df = df.drop('composition_obj', axis=1)

Null compositions 0


/usr/local/lib/python3.12/dist-packages/matminer/featurizers/composition/orbital.py:115: UserWarning:

ValenceOrbital(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.

/usr/local/lib/python3.12/dist-packages/matminer/utils/data.py:326: UserWarning:

MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after f

ValenceOrbital:   0%|          | 0/461 [00:00<?, ?it/s]

##Mulliken electronegativity

In [210]:
from pymatgen.core.periodic_table import Element

def get_mulliken_en(element_symbol):
    el = Element(element_symbol)
    IE = el.ionization_energies[0]  # First ionization energy in eV
    EA = el.electron_affinity       # Electron affinity in eV

    if IE is None or EA is None:
        return None

    return (IE + EA) / 2


def calc_average_electronegativity(formula):
  # Example: For Fe2O3
  comp = Composition(formula)
  # Weighted mean electronegativity
  total_atoms = comp.num_atoms
  mean_en = sum(
    #comp[el] / total_atoms * Element(el).X
    comp[el] / total_atoms * get_mulliken_en(el)
    for el in comp.elements)
  return mean_en

In [211]:
df["Average Mulliken electronegativity"] = df["Hill formula"].apply(calc_average_electronegativity)

##Valence electron density

In [212]:
def split_formula(formula):
  output={}
  elements_with_indexes = formula.split()
  for el in elements_with_indexes:
    #match = re.match(r"([A-Za-z]+)(\d+)$", el)
    match = re.match(r"([A-Za-z]+)(\d+(?:\.\d+)?)$", el)
    if match:
      output[match.group(1)]=float(match.group(2))
    else:
      output[ el]=float(1)
  return output

def get_valence_electrons_number(hill_fomula):
  split = split_formula(hill_fomula)
  print(split)
  v = split.get("O")
  if v == None:
    return 0
  else:
    return 2*v

In [213]:
df["Valence electrons"] = df["Hill formula"].apply(get_valence_electrons_number)

{'O': 2.0, 'Ti': 1.0}
{'O': 2.0, 'Ti': 1.0}
{'La': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'O': 7.0, 'Pr': 1.0, 'Rb': 1.0, 'Ta': 2.0}
{'Nd': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'O': 7.0, 'Rb': 1.0, 'Sm': 1.0, 'Ta': 2.0}
{'O': 5.0, 'Ta': 2.0}
{'Nb': 2.0, 'O': 5.0}
{'La': 4.0, 'O': 12.0, 'Ti': 3.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'La': 4.0, 'O': 15.0, 'Sr': 1.0, 'Ti': 4.0}
{'Ba': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'Ba': 3.0, 'La': 1.0, 'Nb': 3.0, 'O': 12.0}
{'Nb': 4.0, 'O': 15.0, 'Sr': 5.0}
{'Ba': 5.0, 'Nb': 4.0, 'O': 15.0}
{'La': 4.0, 'O': 12.0, 'Ti': 3.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'La': 4.0, 'O': 15.0, 'Sr': 1.0, 'Ti': 4.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'K': 4.0, 'Nb': 6.0, 'O': 17.0}
{'K': 4.0, 'Nb': 4.0, 'O': 17.0, 'Ta': 2.0}
{'K': 4.0, 'Nb': 3.0, 'O': 17.0, 'Ta': 3.0}
{'K': 4.0, 'Nb': 2.0, 'O': 17.0, 'Ta': 4.0}
{'K': 4.0, 'O': 17.0, 'Ta': 6.0}
{'Nb': 6.0, 'O': 17.0, 'Rb': 4.0}
{'Nb': 4.0, 'O': 17.0, 'Rb': 4.0, 'Ta': 2.0}
{'Nb': 3

##CIF files descriptors

In [214]:
from pymatgen.core.structure import Structure

def getCellVolume(ID):
  print("getCellVolume() ID:",ID)
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0

  if(structure == None):
    return 0
  return structure.volume

In [215]:
def getCellZValue(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0
  if(structure == None):
    return 0
  return structure.composition.num_atoms / structure.composition.reduced_composition.num_atoms

In [216]:
def abcExtractionFromMP(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return [0,0,0]
  else:
    return  [0,0,0]

  if(structure == None):
    return  [0,0,0]
  a = structure.lattice.a
  b = structure.lattice.b
  c = structure.lattice.c
  return [a,b,c]

In [217]:
CIF_source = "MP_CIF_modified"
if(extract_CIFs):
  CIF_source = "Materials Project ID"

df["Volume_MP"]=df[CIF_source].apply(getCellVolume)
df["Z_MP"]=df[CIF_source].apply(getCellZValue)
df['Valence Electrons Density_MP'] = (df['Valence electrons'] * df['Z_MP']) / df['Volume_MP']
df[['a_MP', 'b_MP', 'c_MP']] = df[CIF_source].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: mp-1245098
getCellVolume() ID: mp-1245098
getCellVolume() ID: mp-541600
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-3249
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-1228245
getCellVolume() ID: mp-1228150
getCellVolume() ID: mp-561133
getCellVolume() ID: mp-3563
getCellVolume() ID: mp-3249
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-1228245
getCellVolume() ID: mp-560692


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: M_MP16
getCellVolume() ID: M_MP17
getCellVolume() ID: M_MP18
getCellVolume() ID: M_MP19
getCellVolume() ID: M_MP20
getCellVolume() ID: M_MP21
getCellVolume() ID: M_MP22
getCellVolume() ID: M_MP23
getCellVolume() ID: M_MP24
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: mp-3614
getCellVolume() ID: mp-10347
getCellVolume() ID: M_MP26
getCellVolume() ID: mp-10347
getCellVolume() ID: M_MP28
getCellVolume() ID: mp-1104930
getCellVolume() ID: M_MP31
getCellVolume() ID: M_MP32
getCellVolume() ID: M_MP33
getCellVolume() ID: M_MP34
getCellVolume() ID: M_MP35
getCellVolume() ID: mp-1104930
getCellVolume() ID: M_MP36
getCellVolume() ID: M_MP37
getCellVolume() ID: M_MP38
getCellVolume() ID: M_MP39
getCellVolume() ID: M_MP40
getCellVolume() ID: mp-990430
getCellVolume() ID: mp-6259
getCellVolume() ID: mp-7148
getCellVolume() ID: M_MP55
getCellVolume() ID: mp-3614
getCellVolume() ID: mp-990430
getCellVolume() ID

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Missing elements K from PMG structure composition



getCellVolume() ID: HLaTiO4_EtNH2
getCellVolume() ID: HLaTiO4_PrNH2
getCellVolume() ID: HLaTiO4_BuNH2
getCellVolume() ID: HLaTiO4_HxNH2
getCellVolume() ID: HLaTiO4_MeOH
getCellVolume() ID: HLaTiO4_EtOH
getCellVolume() ID: HLaTiO4_PrOH
getCellVolume() ID: HLaTiO4_BuOH
getCellVolume() ID: HLaTiO4_HxOH
getCellVolume() ID: M_MP223
getCellVolume() ID: M_MP224
getCellVolume() ID: M_MP225
getCellVolume() ID: M_MP226
getCellVolume() ID: M_MP227
getCellVolume() ID: M_MP228
getCellVolume() ID: M_MP229
getCellVolume() ID: M_MP230
getCellVolume() ID: M_MP231
getCellVolume() ID: M_MP232
getCellVolume() ID: M_MP233
getCellVolume() ID: M_MP235
getCellVolume() ID: HLaTiO4_MeNH2
getCellVolume() ID: HLaTiO4_EtNH2
getCellVolume() ID: HLaTiO4_PrNH2
getCellVolume() ID: HLaTiO4_BuNH2
getCellVolume() ID: HLaTiO4_HxNH2
getCellVolume() ID: HLaTiO4_MeOH
getCellVolume() ID: HLaTiO4_EtOH
getCellVolume() ID: HLaTiO4_PrOH
getCellVolume() ID: HLaTiO4_BuOH
getCellVolume() ID: HLaTiO4_HxOH
getCellVolume() ID: M_MP236


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Missing elements K from PMG structure composition

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local

In [218]:
CIF_source = "COD_CIF_modified"
if(extract_CIFs):
  CIF_source = "COD_ID"

df["Volume_COD"]=df[CIF_source].apply(getCellVolume)
df["Z_COD"]=df[CIF_source].apply(getCellZValue)
df['Valence Electrons Density_COD'] = (df['Valence electrons'] * df['Z_COD']) / df['Volume_COD']
df[['a_COD', 'b_COD', 'c_COD']] = df[CIF_source].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: 1010942
getCellVolume() ID: 1010942
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526720
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526777
getCellVolume() ID: -1
getCellVolume() ID: 8101224
getCellVolume() ID: 7039788
getCellVolume() ID: 1526720
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526777
getCellVolume() ID: 1001842
getCellVolume() ID: M_COD12


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: M_COD13
getCellVolume() ID: M_COD14
getCellVolume() ID: M_COD15
getCellVolume() ID: M_COD16
getCellVolume() ID: M_COD17
getCellVolume() ID: M_COD18
getCellVolume() ID: M_COD19
getCellVolume() ID: M_COD20
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: 2102087
getCellVolume() ID: 1522043
getCellVolume() ID: nan
getCellVolume() ID: 1522043
getCellVolume() ID: -1
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: M_COD21
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: 2229871
getCellVolume() ID: M_COD22
getCellVolume() ID: -1
getCellVolume() ID: 7221013
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 2102087
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 2102087
getCellVolume(

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
Pauling file corrections applied.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_

getCellVolume() ID: nan
getCellVolume() ID: sd_1045310
getCellVolume() ID: sd_0548135
getCellVolume() ID: nan
getCellVolume() ID: 1011128
getCellVolume() ID: 1001022
getCellVolume() ID: 1001022
getCellVolume() ID: 2106523
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: M_COD100
getCellVolume() ID: M_COD101
getCellVolume() ID: M_COD102
getCellVolume() ID: M_COD10

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist

In [219]:
CIF_source = "Springer_CIF_modified"
if(extract_CIFs):
  CIF_source = "Springer_ID"

df["Volume_Springer"]=df[CIF_source].apply(getCellVolume)
df["Z_Springer"]=df[CIF_source].apply(getCellZValue)
df['Valence Electrons Density_Springer'] = (df['Valence electrons'] * df['Z_Springer']) / df['Volume_Springer']
df[['a_Springer', 'b_Springer', 'c_Springer']] = df[CIF_source].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: M_Springer6
getCellVolume() ID: sd_1050391
getCellVolume() ID: M_Springer7
getCellVolume() ID: M_Springer8
getCellVolume() ID: sd_1232960
getCellVolume() ID: sd_0314261
getCellVolume() ID: nan
getCellVolume() ID: sd_0306542
getCellVolume() ID: sd_0306445
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: sd_0306542


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_gro

getCellVolume() ID: sd_0306445
getCellVolume() ID: sd_0306542
getCellVolume() ID: sd_0376709
getCellVolume() ID: M_Springer9
getCellVolume() ID: M_Springer10
getCellVolume() ID: M_Springer11
getCellVolume() ID: M_Springer12
getCellVolume() ID: M_Springer13
getCellVolume() ID: M_Springer14
getCellVolume() ID: M_Springer15
getCellVolume() ID: M_Springer16
getCellVolume() ID: M_Springer17
getCellVolume() ID: sd_0314261
getCellVolume() ID: sd_0308503
getCellVolume() ID: sd_1127671
getCellVolume() ID: sd_1232960
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: sd_1200159
getCellVolume() ID: nan
getCellVolume() ID: nan


/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1041: UserWarning:

OH parsed as 

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
OH parsed as 
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
OH parsed as 
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.



getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: sd_1810747
getCellVolume() ID: M_Springer32
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: nan
getCellVolume() ID: sd_1810747


...

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
10 fractional coordinates rounded to ideal values to avoid issues with finite precision.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1041: UserWarning:

OH parsed as 

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
OH parsed as 
No _symmetry_equiv_pos_as_xyz type 

ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412


/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:286: UserWarning:

Possible issue in CIF file at line: ?

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1044: UserWarning:

t parsed as T

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 3 in CIF.
could not convert string to float: 't'

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 4 in CIF.
could not convert string to float: 't'

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
t parsed as T
No structure parsed for section 3 in CIF.
could not c

ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid s

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.



In [220]:
df["Volume_manual"]=df["a, A"]*df["b, A"]*df["c, A"]
df['Valence Electrons Density_manual'] = (df['Valence electrons'] * df['Z']) / df['Volume_manual']

##Oxygen concentration

In [221]:
def count_oxigen(formula):
  #match = re.search(r'O(\d+)', hill_formula)
  #if match:
  #  number = int(match.group(1))
  #  print(number)
  #  return number
  #else:
  #  print("No match found")
  if formula is None:
    return 0
  print(formula)
  comp = Composition(formula)

  # Get number of oxygen atoms
  oxygen_count = comp.get_el_amt_dict().get("O", 0)
  print(oxygen_count)
  return oxygen_count

In [222]:
df['Oxygen_count']=df['Hill formula'].apply(count_oxigen)

O2 Ti
2.0
O2 Ti
2.0
La O7 Rb Ta2
7.0
O7 Pr Rb Ta2
7.0
Nd O7 Rb Ta2
7.0
O7 Rb Sm Ta2
7.0
O5 Ta2
5.0
Nb2 O5
5.0
La4 O12 Ti3
12.0
Ca La4 O15 Ti4
15.0
La4 O15 Sr Ti4
15.0
Ba La4 O15 Ti4
15.0
Ba3 La Nb3 O12
12.0
Nb4 O15 Sr5
15.0
Ba5 Nb4 O15
15.0
La4 O12 Ti3
12.0
Ca La4 O15 Ti4
15.0
La4 O15 Sr Ti4
15.0
Ca La4 O15 Ti4
15.0
K4 Nb6 O17
17.0
K4 Nb4 O17 Ta2
17.0
K4 Nb3 O17 Ta3
17.0
K4 Nb2 O17 Ta4
17.0
K4 O17 Ta6
17.0
Nb6 O17 Rb4
17.0
Nb4 O17 Rb4 Ta2
17.0
Nb3 O17 Rb4 Ta3
17.0
Nb2 O17 Rb4 Ta4
17.0
O17 Rb4 Ta6
17.0
Nb2 O5
5.0
K Nb O3
3.0
K Nb3 O8
8.0
O5 Ta2
5.0
K O3 Ta
3.0
Ca2 Cs O10 Ta3
10.0
Ca2 Cs H Na O10 Ta3
10.0
Ca2 Cs O10 Ta3
10.0
Ca2 Cs H Na O10 Ta3
10.0
K La O4 Ti
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Zr
4.0
K La O4 Ti
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Zr
4.0
H2 O7 Sr Ta2
7.0
Li2 O7 Sr Ta2
7.0
K2 O7 Sr Ta2
7.0
O7 Rb2 Sr Ta2
7.0
K O3 Ta
3.0
H2 O7 Sr Ta2
7.0
K2 O7 Sr Ta2
7.0
K O3 Ta
3.0
K6 La2 O21 T

In [223]:
df['Oxygen_concentration_manual']=df['Z']*df['Oxygen_count']/df['Volume_manual']
df['Oxygen_concentration_MP']=df['Z_MP']*df['Oxygen_count']/df['Volume_MP']
df['Oxygen_concentration_COD']=df['Z_COD']*df['Oxygen_count']/df['Volume_COD']
df['Oxygen_concentration_Springer']=df['Z_Springer']*df['Oxygen_count']/df['Volume_Springer']

##Packing fraction


In [224]:
ionic_radii={"H+":0.02}
difficult_compunds_oxidation_states={
    "Ca Cs Na O10 Ta3":{"Ca":2,"Cs":1,"Na":1,"O":-2,"Ta":5},
    "Fe K2 La2 O10 Ti2":{"Fe":3, "K": 1,"La":3,"O":-2,"Ti":4},
    'Ca2 H Nb O10 Ta':{"Ca":2,"H":1,"Nb": 5,"Ta":5,"O":-2},
    'K2 O10 Sr Ta3':{"K":1,"O":-2,"Sr":2,"Ta":5},
    'H Nb O10 Sr2 Ta':{"H":1,"Nb":5,"O":-2,"Sr":2,"Ta":5},
}

In [225]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
def get_packing_fraction_from_cif(mp_id):
    print(mp_id)
    cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"
    cif_path = os.path.join(cif_folder, f"{mp_id}.cif")

    if not os.path.isfile(cif_path):
        return np.nan  # Return NaN if file doesn't exist
    try:
      structure = Structure.from_file(cif_path)
    except:
      return 0
    comp = structure.composition
    #comp_oxi = comp.oxidation_state_guesses()
    #print(comp_oxi)


    # Extract radius for each site
    site_radii = []

    try:
      print('Calculating oxidation states')
      structure.add_oxidation_state_by_guess()
    except:
      print('Error')
      return


    for site in structure:
        #el = Element(site.species_string)
        try:
          specie = site.specie
        except AttributeError:
          return 0
        print(site.specie.ionic_radii)
        print(site.specie.oxi_state)
        print('Specie ionic radii ',site.specie.ionic_radii)
        print('Site oxi satte ',site.specie.oxi_state)
        rad =site.specie.ionic_radii.get(site.specie.oxi_state)
        print('got site')
        if(rad is not None):
          site_radii.append(rad)

    # Compute atomic volume per site
    atom_volumes = [(4/3) * np.pi * r**3 for r in site_radii]
    total_atom_volume = sum(atom_volumes)

    # Compute packing fraction
    packing_fraction = total_atom_volume / structure.volume
    print('OK')
    return packing_fraction
    #except:
        #return np.nan  # Return NaN if anything fails (e.g., missing radii)

In [226]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
from pymatgen.core.periodic_table import Species

unresolved_compunds = []

def get_packing_fraction_from_formula_and_cell_volume(hill_formula, V, Z):
    if V==0 or np.isnan(V) or Z==0 or np.isnan(Z):
        return np.nan
    print("Enter!")
    print(hill_formula)
    print(V)
    print(Z)
    comp = Composition(hill_formula)
    oxi_guesses = comp.oxi_state_guesses()
    if(len(oxi_guesses)==0):
      oxi_guesses = difficult_compunds_oxidation_states.get(hill_formula)
      if(oxi_guesses==None):
        unresolved_compunds.append(hill_formula)
        return np.nan
    else:
      oxi_guesses = oxi_guesses[0]

    V_ions_formula=0
    for el, amt in comp.items():
        symbol = el.symbol
        el_oxidation_state = oxi_guesses[symbol]
        #element = Element(el)
        print("Element:", symbol)
        print("Element ox state:", el_oxidation_state)
        specie = Species(symbol,el_oxidation_state)
        r = specie.ionic_radius
        if(r==np.nan or r==None):
          ion_formula = symbol
          if(el_oxidation_state!=0 and el_oxidation_state!=1 and el_oxidation_state!=-1):
             ion_formula =  ion_formula+str(el_oxidation_state)
          if(el_oxidation_state>0):
            ion_formula =  ion_formula+str("+")
          if(el_oxidation_state<0):
            ion_formula =  ion_formula+str("-")
          print("Local ionic radii table request for ",ion_formula)
          r= ionic_radii.get(ion_formula)
          if(r==None):
            r=0
        print("r: ", r)
        V_ion = (4/3) * np.pi * r**3
        V_ions_formula += (V_ion*amt)
        print(el," ",amt)

    packing_fraction= Z*V_ions_formula/V
    return packing_fraction
    print("Output!")

In [227]:
#fr = get_packing_fraction_from_formula_and_cell_volume("H2 La2 Ti3 O10",390.7,2)
#print("Fraction: ", fr)

#fr = get_packing_fraction_from_formula_and_cell_volume("O2 Ti",1183,30)
#print("Fraction: ", fr)

fr = get_packing_fraction_from_formula_and_cell_volume("Ca Cs Na O10 Ta3",228,1)
print("Fraction: ", fr)

Enter!
Ca Cs Na O10 Ta3
228
1
Element: Ca
Element ox state: 2
r:  1.14 ang
Ca   1.0
Element: Cs
Element ox state: 1
r:  1.81 ang
Cs   1.0
Element: Na
Element ox state: 1
r:  1.16 ang
Na   1.0
Element: O
Element ox state: -2
r:  1.26 ang
O   10.0
Element: Ta
Element ox state: 5
r:  0.78 ang
Ta   3.0
Fraction:  0.5584979567677797


In [228]:
#df["MP_packing_fraction"] = df["Materials Project ID"].apply(get_packing_fraction_from_cif)
#df["COD_packing_fraction"] = df["COD_ID"].apply(get_packing_fraction_from_cif)
#df["Springer_packing_fraction"] = df["Springer_ID"].apply(get_packing_fraction_from_cif)

df["MP_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_MP"], row["Z_MP"]), axis=1)
df["COD_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_COD"], row["Z_COD"]), axis=1)
df["Springer_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_Springer"], row["Z_Springer"]), axis=1)
df["Manual_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_manual"], row["Z"]), axis=1)

Enter!
O2 Ti
1182.7412601085477
30.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   2.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
O2 Ti
1182.7412601085477
30.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   2.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
La O7 Rb Ta2
173.01142785646587
1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Rb
Element ox state: 1.0
r:  1.66 ang
Rb   1.0
Element: Ta
Element ox state: 5.0
r:  0.78 ang
Ta   2.0
Enter!
La4 O12 Ti3
238.90888175434102
1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   4.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   12.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   3.0
Enter!
Ba La4 O15 Ti4
623.8123076220189
2.0
Element: Ba
Element ox state: 2.0
r:  1.49 ang
Ba   1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   4.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   15.0
Element: T

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for H+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for Ta4.33+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for Zr2+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for N+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for Ta4.67+!



Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Enter!
La2 O7 Ti2
288.3342929018237
2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Enter!
La2 O7 Ti2
288.3342929018237
2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Enter!
La2 O7 Ti2
288.3342929018237
2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Enter!
La2 O7 Ti2
288.3342929018237
2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti  

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C2-!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C2.33-!



Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   4.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   11.0
Element: Nd
Element ox state: 3.0
r:  1.123 ang
Nd   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   5.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
C6 H15 Nd O5 Ti
411.19834477643286
2.0
Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   6.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   15.0
Element: Nd
Element ox state: 3.0
r:  1.123 ang
Nd   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   5.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
H La O4 Ti
203.79933701026056
2.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   4.0
Element

...

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C2-!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C2.33-!



Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   4.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   11.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   5.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
C6 H15 La O5 Ti
774.9130400000001
4.0
Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   6.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   15.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   5.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
H Nd O4 Ti
163.86304
2.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   1.0
Element: Nd
Element ox state: 3.0
r:  1.123 ang
Nd   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   4.0
Element: Ti
Eleme

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C1.33-!



Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   8.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   19.0
Element: Ca
Element ox state: 2.0
r:  1.14 ang
Ca   2.0
Element: Nb
Element ox state: 5.0
r:  0.78 ang
Nb   3.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   11.0
Enter!
Ca2 K Nb3 O10
438.04823999999996
2.0
Element: Ca
Element ox state: 2.0
r:  1.14 ang
Ca   2.0
Element: K
Element ox state: 1.0
r:  1.52 ang
K   1.0
Element: Nb
Element ox state: 5.0
r:  0.78 ang
Nb   3.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   10.0
Enter!
Ca2 H Nb3 O10
233.4784
1.0
Element: Ca
Element ox state: 2.0
r:  1.14 ang
Ca   2.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   1.0
Element: Nb
Element ox state: 5.0
r:  0.78 ang
Nb   3.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   10.0
Enter!
C H5 Ca2 Nb3 O11
311.40164
1.0
Element: C
Element ox state: -2.0
Local ionic radii t

/tmp/ipython-input-1623332607.py:32: UserWarning:

No default ionic radius for Mn4+. Using ls data.



In [229]:
print(unresolved_compunds)

['Ca2 K Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 K Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh']


##Averaging


In [230]:
def averaging_valence_electron_density(MP,COD,Springer):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  print('Count: ',count)
  if count==0:
    return 0
  return sum/count


def averaging_4(MP,COD,Springer,manual):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  print(manual)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  if(pd.notna(manual)):
    sum+=manual
    count+=1
  print('Count: ',count)
  if count==0:
    print("No value")
    return np.nan
  return sum/count

In [231]:
df['Valence Electrons Density avg'] = df.apply(lambda x: averaging_4(x['Valence Electrons Density_MP'], x['Valence Electrons Density_COD'],x['Valence Electrons Density_Springer'],x['Valence Electrons Density_manual']), axis=1)
df['Oxygen_concentration avg'] = df.apply(lambda x: averaging_4(x['Oxygen_concentration_MP'], x['Oxygen_concentration_COD'],x['Oxygen_concentration_Springer'],x['Oxygen_concentration_manual']), axis=1)
df['Packing fraction avg'] = df.apply(lambda x: averaging_4(x['MP_packing_fraction'], x['COD_packing_fraction'],x['Springer_packing_fraction'],x['Manual_packing_fraction']), axis=1)

0.10145921517017749
0.12273338948743877
nan
nan
Count:  2
0.10145921517017749
0.12273338948743877
nan
nan
Count:  2
0.08091951019336545
nan
0.08508481120279507
0.083414433015833
Count:  3
nan
nan
0.08508481120279507
0.08345739139442625
Count:  2
nan
nan
0.08508481120279507
0.08526808257143804
Count:  2
nan
nan
0.08508481120279507
0.08536881551918019
Count:  2
nan
nan
0.12032039226005925
nan
Count:  1
nan
nan
0.10356693154735219
nan
Count:  1
0.10045670895014315
0.10245478442963495
nan
nan
Count:  2
nan
nan
0.10275485551073929
nan
Count:  1
nan
nan
0.10162814320920892
nan
Count:  1
0.09618277688159252
0.09952839965001856
nan
nan
Count:  2
0.08721687109487106
nan
nan
nan
Count:  1
0.09000833274200813
0.09396027924991113
nan
nan
Count:  2
0.08487580957467307
0.08752221672887435
nan
nan
Count:  2
0.10045670895014315
0.10245478442963495
nan
nan
Count:  2
nan
nan
0.10275485551073929
nan
Count:  1
nan
nan
0.10162814320920892
nan
Count:  1
0.09618277688159252
0.09952839965001856
0.102754855510

In [232]:
df['Log_rate'] = np.log(df['Rate, umol/(g*h)'])
df['Log_rate'].replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=["Log_rate"])
df['Log_rate'].value_counts()

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

/tmp/ipython-input-830342094.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,count
Log_rate,
1.609438,8
1.791759,7
3.784190,5
6.253829,5
2.302585,4
...,...
5.091463,1
11.957611,1
11.744037,1


In [233]:
#raise SystemExit("Stopping execution")
print(df.shape[0])
df = df[df["Sac Agent 1"].isna()]
df = df[df["Sac agent 2"].isna()]
print(df.shape[0])

444
427


##CIF embedings

In [253]:
from pymatgen.core import Structure
from matminer.featurizers.site import CrystalNNFingerprint
from matminer.featurizers.structure import SiteStatsFingerprint
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation
from matminer.featurizers.structure import JarvisCFID



In [272]:
jarvis = JarvisCFID()
site_fp = CrystalNNFingerprint.from_preset("ops")
structure_featurizer = SiteStatsFingerprint(site_featurizer=site_fp)

def extract_embedings_from_CIF(cif_file):
  cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"
  cif_path = os.path.join(cif_folder, f"{cif_file}.cif")
  structure = Structure.from_file(cif_path)

  embedding = structure_featurizer.featurize(structure)

  #embedding = jarvis.featurize(structure)
  print(len(embedding))
  print(embedding)

In [273]:
extract_embedings_from_CIF("mp-1245098")
extract_embedings_from_CIF("M_MP98")

122
[np.float64(0.03464571605818764), np.float64(0.04055781150400175), np.float64(0.03464571605818764), np.float64(0.04055781150400175), np.float64(0.3450393327534303), np.float64(0.4071565268980176), np.float64(0.0439803295745255), np.float64(0.1500885393303358), np.float64(0.09778339698913983), np.float64(0.21542002095628437), np.float64(0.15143007571476647), np.float64(0.24809171171420052), np.float64(0.17155381304700443), np.float64(0.29483714457060217), np.float64(0.06332804199688877), np.float64(0.14906769609794668), np.float64(0.28024430335658723), np.float64(0.3484402970686268), np.float64(0.1299832539870134), np.float64(0.2070930227988271), np.float64(0.13696171428793213), np.float64(0.2124572850018789), np.float64(0.1171094966368662), np.float64(0.1612473081617785), np.float64(0.10358172907939675), np.float64(0.22400579351096747), np.float64(0.041878974786184735), np.float64(0.08872669369307055), np.float64(0.06586754786066015), np.float64(0.16764848701808857), np.float64(0.0

In [274]:
#def get_embedings()
df["embedding"] = [None] * len(df)


for idx, row in df.iterrows():
    cif_ID = 0
    if(row['Materials Project verification']):
      cif_ID = row['MP_CIF_modified']
    elif(row['COD verification']):
      cif_ID = row['COD_CIF_modified']
    elif(row['Springer verification']):
      cif_ID = row['Springer_CIF_modified']
    else:
      raise SystemExit("Stopping execution")
    print('CIF: ', cif_ID)
    cif_path = os.path.join(cif_folder, f"{cif_ID}.cif")
    structure = Structure.from_file(cif_path)

    embedding= [0.0] * 1557

    if structure.is_ordered:
        #structure.add_oxidation_state_by_guess()  # improves CrystalNN
        embedding = structure_featurizer.featurize(structure)
        df.at[idx, "embedding"] = embedding
        print(embedding)
    else:
       # df.at[idx, "disordered_structure"] = True
        df.at[idx, "embedding"] = None  # cannot featurize
        print(f"Skipped structure-based features for {cif_ID} (disordered)")


CIF:  mp-1245098


/usr/local/lib/python3.12/dist-packages/pymatgen/analysis/local_env.py:4021: UserWarning:

No oxidation states specified on sites! For better results, set the site oxidation states in the structure.

/usr/local/lib/python3.12/dist-packages/pymatgen/analysis/local_env.py:4023: UserWarning:

No oxidation states specified on sites! For better results, set the site oxidation states in the structure.

/usr/local/lib/python3.12/dist-packages/matminer/featurizers/site/fingerprint.py:450: UserWarning:

CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.



[np.float64(0.03464571605818764), np.float64(0.04055781150400175), np.float64(0.03464571605818764), np.float64(0.04055781150400175), np.float64(0.3450393327534303), np.float64(0.4071565268980176), np.float64(0.0439803295745255), np.float64(0.1500885393303358), np.float64(0.09778339698913983), np.float64(0.21542002095628437), np.float64(0.15143007571476647), np.float64(0.24809171171420052), np.float64(0.17155381304700443), np.float64(0.29483714457060217), np.float64(0.06332804199688877), np.float64(0.14906769609794668), np.float64(0.28024430335658723), np.float64(0.3484402970686268), np.float64(0.1299832539870134), np.float64(0.2070930227988271), np.float64(0.13696171428793213), np.float64(0.2124572850018789), np.float64(0.1171094966368662), np.float64(0.1612473081617785), np.float64(0.10358172907939675), np.float64(0.22400579351096747), np.float64(0.041878974786184735), np.float64(0.08872669369307055), np.float64(0.06586754786066015), np.float64(0.16764848701808857), np.float64(0.04011

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_gro

[np.float64(0.14370303525941086), np.float64(0.2442221588507033), np.float64(0.14370303525941086), np.float64(0.2442221588507033), np.float64(0.19431449277848675), np.float64(0.2142996536571149), np.float64(3.946619094928328e-08), np.float64(5.220884003247159e-08), np.float64(6.333016921214567e-05), np.float64(8.377553474190923e-05), np.float64(0.005264436485721072), np.float64(0.006960311406274996), np.float64(0.15427893659710223), np.float64(0.19665725074207607), np.float64(0.11335029680236279), np.float64(0.12907658964451063), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.20149912654357174), np.float64(0.26655828911551244), np.float64(0.09327034004472945), np.float64(0.12338506222839155), np.float64(0.06970194829438761), np.float64(0.09220701054181617), np.float64(0.1220262132043242), np.float64(0.161425506784794), np.float64(0.138084751751842), np.float64(0.18266895649273227), np.

...

Skipped structure-based features for sd_1810747 (disordered)
CIF:  M_Springer32
Skipped structure-based features for M_Springer32 (disordered)
CIF:  mp-5308
[np.float64(0.05425338396493527), np.float64(0.07125470100707533), np.float64(0.05425338396493527), np.float64(0.07125470100707533), np.float64(0.1848281073367689), np.float64(0.19626394270194208), np.float64(0.014210865798719245), np.float64(0.018580163219424045), np.float64(0.042201486280529193), np.float64(0.06414093829346253), np.float64(0.04359484216946189), np.float64(0.06301595888718084), np.float64(0.10546448449817022), np.float64(0.1609519784295555), np.float64(0.07627478405916852), np.float64(0.1535276078966036), np.float64(0.35298802467306173), np.float64(0.31093834318786984), np.float64(0.11709421004122814), np.float64(0.12452094676905671), np.float64(0.18467749411065426), np.float64(0.18002132251328437), np.float64(0.17184163133352384), np.float64(0.14058664479815208), np.float64(0.12551281620482155), np.float64(0.2230

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.



[np.float64(0.06547759601395403), np.float64(0.09930211709954068), np.float64(0.06547759601395403), np.float64(0.09930211709954068), np.float64(0.19422023979174324), np.float64(0.21503874252827143), np.float64(3.664166142083967e-08), np.float64(4.874014262151412e-08), np.float64(6.044803237556206e-05), np.float64(8.014540042719372e-05), np.float64(0.005129673707523987), np.float64(0.006787720111790811), np.float64(0.1548214596492221), np.float64(0.19776024321455235), np.float64(0.11331331227918141), np.float64(0.12744289538054263), np.float64(0.1345750702638005), np.float64(0.28512129315703433), np.float64(0.12121712972483006), np.float64(0.2570631217630212), np.float64(0.06680514393656514), np.float64(0.1415817446600846), np.float64(0.014734156154130396), np.float64(0.03104822836177083), np.float64(0.2896734707989966), np.float64(0.338420494259437), np.float64(0.10046301081365999), np.float64(0.11928210109195446), np.float64(0.15683979427936734), np.float64(0.27542924706403277), np.fl

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Missing elements K from PMG structure composition



[np.float64(0.4896003955716619), np.float64(0.4060753658035494), np.float64(0.4896003955716619), np.float64(0.4060753658035494), np.float64(0.0820231006946059), np.float64(0.12458925477885173), np.float64(0.0001801836213191749), np.float64(0.0003928028419676293), np.float64(0.002020593799272206), np.float64(0.004410511818966065), np.float64(0.008405263812599229), np.float64(0.015297707211025491), np.float64(0.0655570052541993), np.float64(0.11492164716658904), np.float64(0.04225222894189925), np.float64(0.06769617114050704), np.float64(0.07667986119205729), np.float64(0.13992127840030683), np.float64(0.02205872102449433), np.float64(0.04727624815791293), np.float64(0.058224998264216975), np.float64(0.12889970779235588), np.float64(0.017538820306479165), np.float64(0.03386492452678127), np.float64(0.18163192235010447), np.float64(0.2883533483688789), np.float64(0.050520846009077545), np.float64(0.10470233813782369), np.float64(0.12888481558453893), np.float64(0.2253082486541347), np.flo

In [275]:
df.to_excel('checkpoint_cif_embeddings.xlsx')

In [278]:
df = df[df["embedding"].notna()]
emb_df = pd.DataFrame(df["embedding"].to_list(), index=df.index)
emb_df.columns = [f"emb_{i}" for i in range(emb_df.shape[1])]
df = pd.concat([df, emb_df], axis=1)

In [288]:
df.drop(columns=["embedding"], inplace=True)

In [280]:
df.to_excel('checkpoint_cif_embeddings_split.xlsx')

In [ ]:
#!pip uninstall ase -y
#!pip install ase==3.22.1
#!pip install m3gnet

In [ ]:
#raise SystemExit()

In [ ]:
#from pymatgen.core import Structure
#from m3gnet.models import M3GNet

#test_cif = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/mp-557195.cif"
#structure = Structure.from_file(test_cif )
#model = M3GNet.load()

#embedding = model.get_structure_embedding(structure)
#print(embedding.shape)

EDA

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376 entries, 19 to 1088
Columns: 208 entries, Perovskite to emb_121
dtypes: bool(5), float64(182), object(21)
memory usage: 601.1+ KB


In [283]:
df.columns

Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A',
       ...
       'emb_112', 'emb_113', 'emb_114', 'emb_115', 'emb_116', 'emb_117',
       'emb_118', 'emb_119', 'emb_120', 'emb_121'],
      dtype='object', length=208)

In [ ]:
columns_EDA = ['Perovskite','Bandgap, eV',
       'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       'frac s valence electrons', 'frac p valence electrons',
       'frac d valence electrons', 'frac f valence electrons',
       'Average Mulliken electronegativity', 'Valence electrons',
       'Valence Electrons Density avg', 'Oxygen_concentration avg',
       'Packing fraction avg','Log_rate']

In [ ]:
categorical_cols = [col for col in columns_EDA
                    if df[col].dtype in ['object', 'category','bool']]

numerical_cols = [col for col in columns_EDA
                  if df[col].dtype in ['int64', 'float64']]
print(categorical_cols)
print(numerical_cols)

In [ ]:
import plotly.express as px
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

def plot_smooth_distribution(df, column):
    data = df[column].dropna().values

    kde = gaussian_kde(data)
    x_vals = np.linspace(data.min(), data.max(), 500)
    y_vals = kde(x_vals)

    # Plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        line=dict(width=3),
        name=f'{column} KDE'
    ))

    fig.update_layout(
        title=f"Distribution of '{column}'",
        xaxis_title=column,
        yaxis_title="Density",
        #template="plotly_white"
    )

    fig.show()

In [ ]:
plot_smooth_distribution(df, 'Bandgap, eV')

In [ ]:
for col in numerical_cols:
    fig = px.histogram(df, x=col, nbins=20, title=f'Histogram of {col}',width=800, height=500)
    fig.show()

In [ ]:
def draw_pie_chart(df, column):
  fig = px.pie(df, names=column, title=f'{column} Distribution',hole=0.3)
  fig.update_traces(textinfo='percent+label')
  fig.show()

In [ ]:
for col in categorical_cols:
     draw_pie_chart(df,col)

In [ ]:
from scipy.stats import pearsonr

def draw_scatter_plot(df,x_column, y_column):
  from sklearn.metrics import r2_score

  x = df[x_column]
  y = df[y_column]

  r, p_value = pearsonr(x, y)

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      mode='markers',
      name='Data',
      marker=dict(size=8, color='blue', opacity=0.7)
  ))

  fig.add_annotation(
    x=np.mean(x),
    y=np.max(y),
    text=f"Pearson r = {r:.3f}",
    showarrow=False,
    font=dict(size=14, color="red")
  )

  fig.update_layout(
      xaxis_title=f'{x_column}',
      yaxis_title=f'{y_column}',
      template="plotly_white",
      width=600,
      height=600,
  )

  fig.show()

In [ ]:
for col in numerical_cols:
  draw_scatter_plot(df,col,'Rate, umol/(g*h)')

In [ ]:
for col in numerical_cols:
  draw_scatter_plot(df,col,target_col)

df_selected = df[numerical_cols]


In [306]:
def draw_corr_heatmap(_df):
  # Compute correlation matrix
  corr_matrix = _df.corr()

  # Plot heatmap
  fig = px.imshow(
      corr_matrix,
      text_auto=True,        # show correlation values
      aspect="auto",
      color_continuous_scale='RdBu_r',  # diverging color map
      title="Correlation Heatmap",
      width=1000,
      height=1000,
  )
  fig.show()

In [305]:
draw_corr_heatmap(df_selected)

NameError: name 'df_selected' is not defined

In [ ]:
df.to_excel('checkpoint_descriptors.xlsx')

#Preparing data for ML

##Categorization


In [ ]:
#raise SystemExit("Stopping execution")

In [289]:
categorical_columns = df.select_dtypes(include='object').columns.tolist()
categorical_columns.remove('Perovskite')
categorical_columns.remove('Interlayer space composition')
categorical_columns.remove('Materials Project ID')
categorical_columns.remove('COD_ID')
categorical_columns.remove('Springer_ID')
categorical_columns.remove('Symmetry group')
categorical_columns.remove('Sac Agent 1')
categorical_columns.remove('Sac agent 2')
categorical_columns.remove('Hill formula')
categorical_columns

['Prep Method',
 'Promotion method',
 'Promoter',
 'Light type',
 'Cell material',
 'MP_CIF_modifier',
 'COD_CIF_modifier',
 'Springer_CIF_modifier',
 'MP_CIF_modified',
 'COD_CIF_modified',
 'Springer_CIF_modified']

In [290]:
df2 = df['Light type'].value_counts()
print(df2.index)

Index(['Hg lamp', 'Xe lamp'], dtype='object', name='Light type')


In [291]:
df[categorical_columns].isna().sum()

,0
Prep Method,0
Promotion method,0
Promoter,0
Light type,0
Cell material,0
MP_CIF_modifier,167
COD_CIF_modifier,341
Springer_CIF_modifier,363
MP_CIF_modified,10
COD_CIF_modified,85


In [294]:
for col in df.columns:
    print(col)

Perovskite
Interlayer space composition
Bandgap, eV
Materials Project ID
COD_ID
Springer_ID
Z
a, A
b, A
c, A
Symmetry group
d,A
Number of octahedrons on a layer
CalcT(K)
Calc time (h)
Nitrogen
Promoter, w%
Surface area, m2/g
CatW, g/L
Alcohol, %
Sac Agent 1
Sac agent 2
Power, W
Wave length min, nm
Temperature, K
Rate, umol/(g*h)
Materials Project verification
COD verification
Springer verification
General verification
Hill formula
avg s valence electrons
avg p valence electrons
avg d valence electrons
avg f valence electrons
frac s valence electrons
frac p valence electrons
frac d valence electrons
frac f valence electrons
Average Mulliken electronegativity
Valence electrons
Volume_MP
Z_MP
Valence Electrons Density_MP
a_MP
b_MP
c_MP
Volume_COD
Z_COD
Valence Electrons Density_COD
a_COD
b_COD
c_COD
Volume_Springer
Z_Springer
Valence Electrons Density_Springer
a_Springer
b_Springer
c_Springer
Volume_manual
Valence Electrons Density_manual
Oxygen_count
Oxygen_concentration_manual
Oxygen_co

In [292]:
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
df.columns


Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A',
       ...
       'Springer_CIF_modified_sd_0376709', 'Springer_CIF_modified_sd_1050391',
       'Springer_CIF_modified_sd_1127671', 'Springer_CIF_modified_sd_1150217',
       'Springer_CIF_modified_sd_1200159', 'Springer_CIF_modified_sd_1210175',
       'Springer_CIF_modified_sd_1232960', 'Springer_CIF_modified_sd_1241784',
       'Springer_CIF_modified_sd_1241787', 'Springer_CIF_modified_sd_1925412'],
      dtype='object', length=589)

In [297]:
df= df.iloc[:, :215]
print(df_first_215.columns)

Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A',
       ...
       'Promoter_CuO', 'Promoter_Fe2O3', 'Promoter_Ni', 'Promoter_NiO',
       'Promoter_No promoter', 'Promoter_Pt', 'Promoter_Rh', 'Promoter_RuO2',
       'Promoter_ZnO', 'Light type_Xe lamp'],
      dtype='object', length=215)


In [298]:
columns_for_ML = df.columns
columns_to_exlude = ['Perovskite', 'Interlayer space composition','Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'Number of octahedrons on a layer','Sac Agent 1',
       'Sac agent 2','Rate, umol/(g*h)','Hill formula','Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Electrons Density_COD', 'a_COD', 'b_COD', 'c_COD',
       'Volume_Springer', 'Z_Springer', 'Valence Electrons Density_Springer',
       'a_Springer', 'b_Springer', 'c_Springer', 'Volume_manual',
       'Valence Electrons Density_manual', 'Oxygen_count',
       'Oxygen_concentration_manual', 'Oxygen_concentration_MP',
       'Oxygen_concentration_COD', 'Oxygen_concentration_Springer',
       'MP_packing_fraction', 'COD_packing_fraction',
       'Springer_packing_fraction', 'Manual_packing_fraction']

columns_for_ML = [x for x in columns_for_ML if x not in columns_to_exlude]
print(columns_for_ML)


['Bandgap, eV', 'd,A', 'CalcT(K)', 'Calc time (h)', 'Nitrogen', 'Promoter, w%', 'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Power, W', 'Wave length min, nm', 'Temperature, K', 'Materials Project verification', 'COD verification', 'Springer verification', 'General verification', 'avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'frac s valence electrons', 'frac p valence electrons', 'frac d valence electrons', 'frac f valence electrons', 'Average Mulliken electronegativity', 'Valence electrons', 'Valence Electrons Density avg', 'Oxygen_concentration avg', 'Packing fraction avg', 'Log_rate', 'emb_0', 'emb_1', 'emb_2', 'emb_3', 'emb_4', 'emb_5', 'emb_6', 'emb_7', 'emb_8', 'emb_9', 'emb_10', 'emb_11', 'emb_12', 'emb_13', 'emb_14', 'emb_15', 'emb_16', 'emb_17', 'emb_18', 'emb_19', 'emb_20', 'emb_21', 'emb_22', 'emb_23', 'emb_24', 'emb_25', 'emb_26', 'emb_27', 'emb_28', 'emb_29', 'emb_30', 'emb_31', 'emb_32', 'emb_33', 'emb_34',

In [299]:
df = df[columns_for_ML]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376 entries, 19 to 1088
Columns: 171 entries, Bandgap, eV to Light type_Xe lamp
dtypes: bool(24), float64(147)
memory usage: 443.6 KB


In [300]:
df["Valence Electrons Density avg"] = df["Valence Electrons Density avg"].fillna(df["Valence Electrons Density avg"].mean())
df["Oxygen_concentration avg"] = df["Oxygen_concentration avg"].fillna(df["Oxygen_concentration avg"].mean())
df["Packing fraction avg"] = df["Packing fraction avg"].fillna(df["Packing fraction avg"].mean())

In [301]:
df["d,A"] = df["d,A"].fillna(df["d,A"].mean())
print(df["d,A"].mean())

15.499487872340428


In [302]:
df.isna().sum().sum()

np.int64(0)

In [307]:
draw_corr_heatmap(df)

In [308]:
features_to_remove=['Oxygen_concentration avg','Valence electrons']
df = df.drop(columns=features_to_remove)

##Normalization

In [309]:
from sklearn.preprocessing import MinMaxScaler

features = list(df.columns)
features.remove('Log_rate')

scaler = MinMaxScaler()

df_normalized = df.copy()
df_normalized[features] = scaler.fit_transform(df_normalized[features])
df_normalized.head()

,"Bandgap, eV","d,A",CalcT(K),Calc time (h),Nitrogen,"Promoter, w%","Surface area, m2/g","CatW, g/L","Alcohol, %","Power, W",...,Promoter_CuO,Promoter_Fe2O3,Promoter_Ni,Promoter_NiO,Promoter_No promoter,Promoter_Pt,Promoter_Rh,Promoter_RuO2,Promoter_ZnO,Light type_Xe lamp
19,0.142857,0.000000,0.678085,0.470028,0.0,0.05,0.098972,0.996462,0.158889,0.866667,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
69,0.785714,0.319540,0.750000,0.245283,0.0,0.00,0.098972,0.341989,0.000000,0.733333,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
70,0.500000,0.319540,0.750000,0.245283,0.0,0.00,0.098972,0.341989,0.000000,0.733333,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
71,0.714286,0.318966,0.750000,0.245283,0.0,0.00,0.098972,0.341989,0.000000,0.733333,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
72,0.714286,0.319253,0.750000,0.245283,0.0,0.00,0.098972,0.341989,0.000000,0.733333,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [310]:
draw_corr_heatmap(df_normalized)

In [311]:
X = df_normalized.drop(columns=[target_col])
y = df_normalized[target_col]

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
threshold = 0.9
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print("Columns to drop:", to_drop)
df_reduced = df.drop(columns=to_drop)

Columns to drop: ['Wave length min, nm', 'Temperature, K', 'frac d valence electrons', 'frac f valence electrons', 'emb_0', 'emb_1', 'emb_2', 'emb_3', 'emb_7', 'emb_8', 'emb_9', 'emb_11', 'emb_17', 'emb_18', 'emb_19', 'emb_20', 'emb_21', 'emb_22', 'emb_23', 'emb_28', 'emb_30', 'emb_32', 'emb_33', 'emb_34', 'emb_35', 'emb_37', 'emb_38', 'emb_39', 'emb_40', 'emb_41', 'emb_42', 'emb_43', 'emb_46', 'emb_47', 'emb_48', 'emb_49', 'emb_50', 'emb_51', 'emb_53', 'emb_54', 'emb_55', 'emb_56', 'emb_57', 'emb_59', 'emb_60', 'emb_61', 'emb_62', 'emb_63', 'emb_65', 'emb_67', 'emb_68', 'emb_69', 'emb_70', 'emb_71', 'emb_73', 'emb_75', 'emb_76', 'emb_77', 'emb_78', 'emb_79', 'emb_81', 'emb_82', 'emb_83', 'emb_84', 'emb_85', 'emb_86', 'emb_87', 'emb_89', 'emb_90', 'emb_91', 'emb_93', 'emb_94', 'emb_95', 'emb_96', 'emb_97', 'emb_99', 'emb_101', 'emb_103', 'emb_105', 'emb_107', 'emb_115', 'Promoter_Pt', 'Promoter_RuO2']


In [312]:
df_normalized.to_excel("checkpoint_ML_input.xlsx")

#ML


In [313]:
#raise SystemExit("Stopping execution")


print('Std:',y.std())

print(X.shape[0])

Std: 3.0355187942597914
376


In [314]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold

In [315]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True
)


In [316]:
def draw_actual_predicted(_y_test, _y_pred):
  fig = px.scatter(
    x=_y_test,
    y=_y_pred,
    labels={'x':'Actual', 'y':'Predicted'},
    title='Actual vs Predicted',
    width = 700,
    height = 700,
  )
  fig.add_shape(
      type="line",
      x0=_y_test.min(), y0=_y_pred.min(),
      x1=_y_test.max(), y1=_y_pred.max(),
      line=dict(color="red", dash="dash")
  )
  rmse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  fig.add_annotation(
    x=0.05, y=0.95, xref="paper", yref="paper",
    text=f"R² = {r2:.4f}<br>RMSE = {rmse:.4f}",
    showarrow=False,
    font=dict(size=14)
  )
  fig.show()

##Linear regression

In [317]:
linear_model = LinearRegression()

if(cross_validation):
  cv = KFold(n_splits=5, shuffle=True)
  scores = cross_val_score(linear_model, X, y, cv=cv, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(linear_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  linear_model.fit(X_train, y_train)
  y_pred = linear_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-2.39911400e+12 -2.86632979e+14 -2.62481192e+15 -1.68703843e+13
 -7.07582549e+11]
Mean RMSE: 14932120.172084486


In [318]:
def estimate_performance(_y_test, _y_pred):
  raisemse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  print("RMSE:", mse)
  print("R² Score:", r2)

##Decision tree regression

In [319]:
from sklearn.tree import DecisionTreeRegressor

In [320]:
tree_model = DecisionTreeRegressor(random_state=42)

if(cross_validation):
  scores = cross_val_score(tree_model, X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(tree_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  tree_model.fit(X_train, y_train)
  y_pred = tree_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-9.3189615  -4.44880685 -5.17848166 -0.76509454 -7.8302621 ]
Mean RMSE: 2.2221004063259766


In [321]:
import matplotlib.pyplot as plt

if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': tree_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

## Random forest

In [322]:
from sklearn.ensemble import RandomForestRegressor

In [323]:
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=3,
    min_samples_split = 5,
    min_samples_leaf = 3,
    max_features=2,
    random_state=42
)

if(cross_validation):
  scores = cross_val_score(rf_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(rf_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  rf_model.fit(X_train, y_train)
  y_pred = rf_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-7.89949464 -3.65510572 -3.07292156 -6.2419493  -8.4712898 ]
Mean RMSE: 2.3768698930578744


In [324]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': rf_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [325]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 5],
    'max_features': [1, 2, 3]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42),
                           param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X,y)
print('Best params',grid_search.best_params_)
print('Best score',grid_search.best_score_)

Best params {'max_depth': 5, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 150}
Best score -4.716748673163575


In [326]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-5.64419949 -3.66355428 -2.29403148 -3.81290453 -8.16905358]
Mean RMSE: 2.1230442865999635


##Gradient boosting

In [327]:
from sklearn.ensemble import GradientBoostingRegressor

In [328]:
gb_model = GradientBoostingRegressor(
    n_estimators=100,      # number of boosting stages
    learning_rate=0.1,     # shrinkage factor
    max_depth=3,           # depth of each tree
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

if(cross_validation):
  scores = cross_val_score(gb_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(gb_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  gb_model.fit(X_train, y_train)
  y_pred = gb_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-5.92582121 -3.27807612 -2.54758772 -0.59461975 -7.85544109]
Mean RMSE: 1.8829670238988379


In [329]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': gb_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [330]:
param_grid = {
    'n_estimators': [100, 150],        # number of boosting stages
    'learning_rate': [0.01, 0.1],   # shrinkage factor
    'max_depth': [2, 3, 4],                # depth of each tree
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 1.0]                # fraction of samples for each tree
}

# Grid search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',          # or 'neg_mean_squared_error'
    n_jobs=-1,
    verbose=2
)

# Fit on training data
grid_search.fit(X, y)

# Best hyperparameters
print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 150, 'subsample': 0.8}
Best CV RMSE: -3.032588201411084


In [331]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-4.75142566 -2.60967384 -1.74102893 -0.65686346 -5.40394912]
Mean RMSE: 1.6499632529487869


##KNN

In [332]:
from sklearn.neighbors import KNeighborsRegressor

In [333]:
knn_model = KNeighborsRegressor(n_neighbors=5)

if(cross_validation):
  scores = cross_val_score(knn_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(knn_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  knn_model.fit(X_train, y_train)
  y_pred = knn_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-5.84298904 -4.09661722 -4.79706979 -1.25934703 -5.1539747 ]
Mean RMSE: 2.0047804328725802


In [334]:
param_grid = {
    'n_neighbors': list(range(1, 21)),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_search = GridSearchCV(
    KNeighborsRegressor(),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X,y)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters: {'n_neighbors': 13, 'p': 1, 'weights': 'distance'}
Best CV RMSE: -3.806230098553871


In [335]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-5.11427704 -3.16724441 -2.27106627 -2.37763376 -6.10092901]
Mean RMSE: 1.9120242953881292


##Catboost

In [336]:
!pip install catboost
from catboost import CatBoostRegressor

In [337]:
cat_boost_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    verbose=100
)

if(cross_validation):
  scores = cross_val_score(cat_boost_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(cat_boost_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  cat_boost_model.fit(X_train, y_train)
  y_pred = cat_boost_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

0:	learn: 2.6888149	total: 96.5ms	remaining: 1m 36s
100:	learn: 0.5936379	total: 2.58s	remaining: 23s
200:	learn: 0.3807623	total: 4.78s	remaining: 19s
300:	learn: 0.2691692	total: 7.82s	remaining: 18.2s
400:	learn: 0.2085449	total: 10.6s	remaining: 15.9s
500:	learn: 0.1697436	total: 12.6s	remaining: 12.5s
600:	learn: 0.1398912	total: 14.6s	remaining: 9.71s
700:	learn: 0.1177371	total: 16.7s	remaining: 7.13s
800:	learn: 0.0997001	total: 18.6s	remaining: 4.63s
900:	learn: 0.0872579	total: 20.4s	remaining: 2.24s
999:	learn: 0.0769456	total: 23.4s	remaining: 0us
0:	learn: 2.9763206	total: 61.3ms	remaining: 1m 1s
100:	learn: 0.6458862	total: 3.42s	remaining: 30.5s
200:	learn: 0.4041854	total: 6.84s	remaining: 27.2s
300:	learn: 0.2852770	total: 9.07s	remaining: 21.1s
400:	learn: 0.2161430	total: 11.6s	remaining: 17.3s
500:	learn: 0.1651083	total: 15s	remaining: 14.9s
600:	learn: 0.1326989	total: 18.2s	remaining: 12.1s
700:	learn: 0.1097996	total: 21.1s	remaining: 9s
800:	learn: 0.0937894	to

In [338]:
param_grid = {
    'depth': [4,6,8],
    'learning_rate':[0.01,0.05,0.1 ]
}

grid_search = GridSearchCV(cat_boost_model, param_grid, cv=5,scoring='neg_mean_squared_error',)
grid_search.fit(X,y)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

0:	learn: 2.8479539	total: 4.52ms	remaining: 4.52s
100:	learn: 1.7175637	total: 837ms	remaining: 7.45s
200:	learn: 1.3214107	total: 1.58s	remaining: 6.28s
300:	learn: 1.1454635	total: 2.42s	remaining: 5.63s
400:	learn: 1.0489910	total: 3.36s	remaining: 5.01s
500:	learn: 0.9818832	total: 4.42s	remaining: 4.4s
600:	learn: 0.9257653	total: 5.67s	remaining: 3.77s
700:	learn: 0.8815852	total: 7.02s	remaining: 2.99s
800:	learn: 0.8416633	total: 8.28s	remaining: 2.06s
900:	learn: 0.8079052	total: 9.99s	remaining: 1.1s
999:	learn: 0.7767408	total: 10.8s	remaining: 0us
0:	learn: 3.1729112	total: 15.4ms	remaining: 15.4s
100:	learn: 1.8817515	total: 1.26s	remaining: 11.2s
200:	learn: 1.3926518	total: 2.24s	remaining: 8.9s
300:	learn: 1.1897458	total: 3.37s	remaining: 7.83s
400:	learn: 1.0842587	total: 4.42s	remaining: 6.6s
500:	learn: 1.0191110	total: 5.28s	remaining: 5.26s
600:	learn: 0.9646098	total: 6.27s	remaining: 4.16s
700:	learn: 0.9208427	total: 7.13s	remaining: 3.04s
800:	learn: 0.884735

In [339]:
best_model = grid_search.best_estimator_

if(cross_validation):
  print("Cross validation")
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Cross validation
0:	learn: 2.8479539	total: 4.08ms	remaining: 4.07s
100:	learn: 1.7175637	total: 318ms	remaining: 2.83s
200:	learn: 1.3214107	total: 632ms	remaining: 2.51s
300:	learn: 1.1454635	total: 973ms	remaining: 2.26s
400:	learn: 1.0489910	total: 1.29s	remaining: 1.92s
500:	learn: 0.9818832	total: 1.66s	remaining: 1.65s
600:	learn: 0.9257653	total: 1.99s	remaining: 1.32s
700:	learn: 0.8815852	total: 2.31s	remaining: 984ms
800:	learn: 0.8416633	total: 2.62s	remaining: 652ms
900:	learn: 0.8079052	total: 2.97s	remaining: 327ms
999:	learn: 0.7767408	total: 3.28s	remaining: 0us
0:	learn: 3.1729112	total: 4.51ms	remaining: 4.51s
100:	learn: 1.8817515	total: 373ms	remaining: 3.32s
200:	learn: 1.3926518	total: 783ms	remaining: 3.11s
300:	learn: 1.1897458	total: 1.16s	remaining: 2.7s
400:	learn: 1.0842587	total: 1.56s	remaining: 2.33s
500:	learn: 1.0191110	total: 1.93s	remaining: 1.92s
600:	learn: 0.9646098	total: 2.62s	remaining: 1.74s
700:	learn: 0.9208427	total: 3.52s	remaining: 1.5s
8

Scores:  [-4.82467562 -3.18634005 -2.40956749 -0.89133353 -6.04489701]
Mean RMSE: 1.787313331581641


In [340]:
y_pred = cross_val_predict(best_model , X, y, cv=5)

0:	learn: 2.8479539	total: 4.15ms	remaining: 4.15s
100:	learn: 1.7175637	total: 366ms	remaining: 3.26s
200:	learn: 1.3214107	total: 711ms	remaining: 2.83s
300:	learn: 1.1454635	total: 1.03s	remaining: 2.39s
400:	learn: 1.0489910	total: 1.39s	remaining: 2.07s
500:	learn: 0.9818832	total: 2.15s	remaining: 2.14s
600:	learn: 0.9257653	total: 2.94s	remaining: 1.95s
700:	learn: 0.8815852	total: 3.8s	remaining: 1.62s
800:	learn: 0.8416633	total: 4.62s	remaining: 1.15s
900:	learn: 0.8079052	total: 5.43s	remaining: 596ms
999:	learn: 0.7767408	total: 5.8s	remaining: 0us
0:	learn: 3.1729112	total: 4.84ms	remaining: 4.83s
100:	learn: 1.8817515	total: 408ms	remaining: 3.63s
200:	learn: 1.3926518	total: 777ms	remaining: 3.09s
300:	learn: 1.1897458	total: 1.17s	remaining: 2.71s
400:	learn: 1.0842587	total: 1.61s	remaining: 2.4s
500:	learn: 1.0191110	total: 1.98s	remaining: 1.97s
600:	learn: 0.9646098	total: 2.39s	remaining: 1.59s
700:	learn: 0.9208427	total: 2.75s	remaining: 1.17s
800:	learn: 0.88473

In [341]:
draw_actual_predicted(y, y_pred)